import modules & 

In [1]:
import pandas as pd
import os
current_directory = os.getcwd()
print("현재 디렉토리:", current_directory)

load data

In [3]:
jpy = pd.read_csv('../dataset/JPY_KRW.csv').drop(['고가','저가','거래량'], axis=1).rename(
    columns={'날짜':'date',
             '종가':'close',
             '시가':'open',
             '변동 %':'close_change'})

In [4]:
jpy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66 entries, 0 to 65
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   date          66 non-null     object 
 1   close         66 non-null     float64
 2   open          66 non-null     float64
 3   close_change  66 non-null     object 
dtypes: float64(2), object(2)
memory usage: 2.2+ KB


In [5]:
# 특수문자 제거, 데이터 타입 변경
jpy['date'] = pd.to_datetime(jpy['date'])

In [6]:
# 날짜순으로 오름차순 정렬
jpy.sort_values(by='date').reset_index(drop=True, inplace=True)

#### 환율 변동값 전처리
* 전일 변동율 = (전날종가-전날시가)/전날시가 yesterday_change <br>
* 야간 변동율 = (당일시가-전날종가)/전날종가 night_change <br>
* 데이터 기간 : 2023/07/31 ~ 2023/10/20

In [7]:
# 파생변수 생성 (전일 변동율 = yesterday_change)
jpy['jpy_yesterday_change'] = round((jpy['close']-jpy['open'])/jpy['open']*100, 2)
jpy['jpy_yesterday_change'] = jpy['jpy_yesterday_change'].shift(1)

# 파생변수 생성(야간 변동율 = night_change)
jpy['close_shift'] = jpy['close'].shift(1)
jpy['jpy_night_change'] = round((jpy['open']-jpy['close_shift'])/jpy['close_shift']*100,2)

jpy.drop('close_shift',axis=1,inplace=True)

In [8]:
# 종가 기준 변동율 전처리 및 데이터 형태 변환( 단위 : % )
jpy['close_change'] = jpy['close_change'].str.replace('%','')
jpy['close_change'] = jpy['close_change'].astype(float)

In [9]:
jpy = jpy.loc[1:,][['date','jpy_yesterday_change','jpy_night_change']]

In [10]:
jpy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65 entries, 1 to 65
Data columns (total 3 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   date                  65 non-null     datetime64[ns]
 1   jpy_yesterday_change  65 non-null     float64       
 2   jpy_night_change      65 non-null     float64       
dtypes: datetime64[ns](1), float64(2)
memory usage: 1.7 KB


csv 파일로 새로 저장

In [11]:
jpy.to_csv("../dataset/jpy_krw_final.csv")